# Sequential Models for Toy Problem 5

How do we train a classifier to count the non-zero values in a sequence of integers?

We have the data ... having already generated data points consisting of features that consist of a sequence of $1$s and $0$ and an output label which is the count of the non-zero features.

In [1]:
import torch
import torch.nn.functional as F
from data_reader import Data

data = Data("data/toy_problem_5_train.txt")

sequence_length = 20
num_output = sequence_length+1
num_hidden = 4

W = torch.nn.Parameter(torch.rand(num_hidden+1, num_hidden))
print("Weights input & hidden to hidden => "+str(W))

b = torch.nn.Parameter(torch.rand(1, num_hidden))
print("Bias to hidden => "+str(b))

V = torch.nn.Parameter(torch.rand(num_hidden, num_output))
print("Weights hidden to output categories => "+str(V))

d = torch.nn.Parameter(torch.rand(1, num_output))
print("Bias to output categories => "+str(d))

optimizer = torch.optim.Adam([W, b, V, d], lr=0.01)

for j in range(1001):
    optimizer.zero_grad()   # zero the gradient buffers
    
    labels, features = data.get_sample(1000)
    
    features = torch.autograd.Variable(torch.Tensor(features))
    #print("Features: "+str(features))
    
    target = torch.autograd.Variable(torch.LongTensor(labels))
    #print("Target: "+str(target))
    
    state = torch.autograd.Variable(torch.zeros(features.size()[0],num_hidden))
    #print("State: "+str(state))

    for i in range(sequence_length):
        features_at_current_step = torch.unsqueeze(features[:,i], 1)
        #print("Features at current step: "+str(features_at_current_step))
        x = torch.cat((state, features_at_current_step), 1)
        state = F.tanh(x.mm(W) + b)
    
    result = state.mm(V) + d
    
    loss = F.cross_entropy(result, target)
    #print("Cross entropy loss: "+str(loss))

    loss.backward()
    
    optimizer.step()
    
    if j % 10 == 0:
        print("The loss is now "+str(loss.data[0]))
    
#print("The first layer weights are now "+str(weights1.data))
#print("\tand the second layer's weights are now "+str(weights2.data))

torch.save(W, "models/toy_problem_5_trained_sequential_deep_model_weights1.bin")
torch.save(b, "models/toy_problem_5_trained_sequential_deep_model_bias1.bin")
torch.save(V, "models/toy_problem_5_trained_sequential_deep_model_weights2.bin")
torch.save(d, "models/toy_problem_5_trained_sequential_deep_model_bias2.bin")

Weights input & hidden to hidden => Parameter containing:
 0.6588  0.5137  0.2802  0.3886
 0.2036  0.6493  0.2917  0.8452
 0.1676  0.8665  0.3543  0.9987
 0.1394  0.8172  0.3156  0.1504
 0.1451  0.5167  0.0110  0.1137
[torch.FloatTensor of size 5x4]

Bias to hidden => Parameter containing:
 0.6514  0.4509  0.4071  0.6924
[torch.FloatTensor of size 1x4]

Weights hidden to output categories => Parameter containing:

Columns 0 to 9 
 0.7931  0.6467  0.1943  0.4167  0.6571  0.1315  0.7598  0.3914  0.2626  0.7024
 0.0998  0.8733  0.9745  0.2042  0.5290  0.1556  0.3832  0.8666  0.3726  0.3792
 0.3009  0.1186  0.2703  0.5419  0.7609  0.3759  0.2803  0.9337  0.6753  0.6684
 0.3307  0.0428  0.7025  0.7086  0.6252  0.1413  0.1440  0.0387  0.9647  0.4536

Columns 10 to 19 
 0.0725  0.8530  0.0865  0.8971  0.6784  0.2744  0.1088  0.8517  0.0380  0.7214
 0.3467  0.1338  0.1561  0.9472  0.6366  0.6784  0.0429  0.7818  0.7486  0.0830
 0.4039  0.2213  0.8707  0.6026  0.0348  0.9106  0.8854  0.8488  0.

The loss has been steadily decreasing.  Testing the model would be a good idea.